In [2]:
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

# Seaborn for plotting and styling
import seaborn as sns

import math
from copy import deepcopy
import h5py
import numpy as np
import random

# build predictive model
from tensorly.regression.kruskal_regression import KruskalRegressor
import tensorly.backend as T
from tensorly.decomposition import parafac

C:\Users\Guruprasad\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using numpy backend.


## Import .MAT files
1. $File1:$ SharedData.mat (raw data)
2. $File2:$ spikeData_1ms_rawSpikes.mat (spikeData binned and referenced according to sessionNum, trialNum and reach-direction)
3. $File3:$ stateData_1ms_cursorState_organized.mat

In [3]:
data_path = "C://Users//Guruprasad//Desktop/OneDrive - California Institute of Technology/Human Neural data analysis/"
file_name = "SharedData.mat"
arrays = {}

g= h5py.File(data_path+file_name, 'r')
for k, v in g.items():
    arrays[k] = np.array(v)
    
arrays.keys()

dict_keys(['#refs#', 'SessionDay', 'sdata', 'state', 'target', 'trialTimes'])

In [4]:
data_path = "C://Users//Guruprasad//Desktop/OneDrive - California Institute of Technology/Human Neural data analysis/"
file_name = "spikeData_1msbins_rawSpikes.mat"

arrays_spikes = {}

f = h5py.File(data_path+file_name,'r') 
for k, v in f.items():
    arrays_spikes[k] = np.array(v)
    
arrays_spikes.keys()

dict_keys(['#refs#', 'struct_rawSpikeData'])

In [5]:
data_path = "C://Users//Guruprasad//Desktop/OneDrive - California Institute of Technology/Human Neural data analysis/Playing with data/"
file_name = "stateData_1msbins_cursorState_organized.mat"
arrays_cursorState = {}

f_cursorState = h5py.File(data_path+file_name,'r') 
for k, v in f_cursorState.items():
    arrays_cursorState[k] = np.array(v)
    
arrays_cursorState.keys()

dict_keys(['#refs#', 'struct_state'])

### Smoothing the neural spiking data

In [6]:
def gaussKernel(windowSize,std_dev):
    
    gauss_kernel = np.zeros([windowSize]);
    
    win_center = (windowSize-1)/2;
    for i in range(windowSize):
        gauss_kernel[i] = math.exp(-math.pow((i-win_center),2)/(2*math.pow(std_dev,2)));
    
    #gauss_kernel = gauss_kernel/np.sum(gauss_kernel); #Normalization
    return gauss_kernel    

def gauss_smooth(a, gauss_kernel):
    
    a_smooth = deepcopy(a);
    
    for row in range(a.shape[0]):
        for col in range(a.shape[1]-len(gauss_kernel)):
            
            a_smooth[row,int(col+(len(gauss_kernel)-1)/2)] = np.mean(a[row,col:col+len(gauss_kernel)]*gauss_kernel);
    
    return a_smooth

In [7]:
#prediction for one session and one trial
s_id = 0
y_ref =arrays['target'][s_id,0]
target_s = g[y_ref][()]
n_trial_s = len(target_s)
y = target_s.squeeze()


### Create Neuron-spikeData tensor

In [8]:
# Index:1 - Number of trials; Index:2 - Number of tasks; Index:3 - Number of sessions
x_ref = arrays_spikes['struct_rawSpikeData']
xstate_ref = arrays_cursorState['struct_state']

# Make a 3-d matrix/tensor using all spikeData over multiple sessions and trials for a single task (reaching direction)

neuron_tempTensor = np.zeros([96,3000,150]);
target_states = np.zeros([3000,2]);

task= 0; #[All 8 reaching directions referenced from 0:7]
ctr = 0; # Keeping track of number of trials (in z-dimension)
gauss_kernel = gaussKernel(20,3);

targets = [];

for sessionNum in range(10):
    for trialNum in range(14):
        
        #trialNum = random.randint(0,7);
        
        rawSpikes = f[x_ref[trialNum,task,sessionNum]];
        
        if len(rawSpikes) <= 2:
            continue;
        else:
            rawSpikes_tr = np.transpose(rawSpikes[0:3000,:]);
            
            #cursorState = f_cursorState[xstate_ref[trialNum, task, sessionNum]];
            
            targets.append(task);
            
            #Normalize data along the rows (so that each neuron has a mean-0)
            mu_matrix = np.mean(rawSpikes_tr,axis=1)
            std_matrix = rawSpikes_tr.std(1)
            
            #print([np.count_nonzero(std_matrix),np.count_nonzero(mu_matrix)])
            #print(std_matrix.squeeze())
            
            # Making zero entry in std_matrix = 1 (to prevent divide by zero run-time error)
        
            zero_index = np.where(std_matrix == 0)[0];
            for index in range(len(zero_index)):
                std_matrix[zero_index[index]] = 1;
            
            #print([np.count_nonzero(std_matrix),np.count_nonzero(mu_matrix)])
                        
            t1 = (rawSpikes_tr.transpose()-mu_matrix).transpose();
            t2 = (t1.transpose()/std_matrix).transpose();
            
            rawSpikes_tr = t2;            
            
            #rawSpikes_tr = gauss_smooth(rawSpikes_tr, gauss_kernel);
            
            
            neuron_tempTensor[:,:,ctr] = rawSpikes_tr;
            ctr = ctr + 1;


#print(ctr)
neuron_tensor = neuron_tempTensor[:,:,0:ctr]

### Import cursor-state 

In [9]:
# Index:1 - Number of trials; Index:2 - Number of tasks; Index:3 - Number of sessions
xstate_ref = arrays_cursorState['struct_state']

cursorState = f_cursorState[xstate_ref[1,task,1]];
target_state = np.transpose(cursorState);

neuron_tensor.shape


(96, 3000, 55)

### Tensor decomposition
CP approximation to kruskal form and via Jenreich's theorem - identify components. 

In [ ]:
# build predictive model
from tensorly.regression.kruskal_regression import KruskalRegressor
import tensorly.backend as T
from tensorly.decomposition import parafac

X = T.tensor(neuron_tensor);
factors = parafac(X,rank=3)


#plt.plot(trialFactors[:,1])

#for comp in factors:
#    comp_e = comp.squeeze()
#    print(comp_e[:,0])

In [ ]:
neuronFactors = factors[0];
temporalFactors = factors[1];
trialFactors = factors[2];

plt.figure(1)
plt.plot(neuronFactors)
plt.title('Neuron Factors')

plt.figure(2)
plt.plot(temporalFactors)
plt.title('temporal Factors')

plt.figure(3)
plt.plot(trialFactors)
plt.title('Trial Factors')
plt.legend({'1','2','3'})

### Tensor regression

In [13]:
train_tensor = neuron_tensor[:,:,:];
print(train_tensor.shape)

test_tensor = neuron_tensor[:,:,35:50];
test_tensor.shape

train_targets = np.array(targets[0:35])
test_targets = np.array(targets[35:50])
train_targets

train_tensor = np.reshape(train_tensor, (train_tensor.shape[1],train_tensor.shape[0], train_tensor.shape[2]))
test_tensor = np.reshape(test_tensor, (test_tensor.shape[1],test_tensor.shape[0], test_tensor.shape[2]))

print(train_tensor.shape)
print(train_targets.shape)

print(target_state.shape)

(96, 3000, 55)
(3000, 96, 55)
(35,)
(3000, 2)


In [14]:
# Tensor regression

X = T.tensor(train_tensor);
y1 = T.tensor(target_state[:,0]);
y2 = T.tensor(target_state[:,1]);

print(X.shape)
print(y1.shape)
rank = 2
estimator1 = KruskalRegressor(weight_rank=rank, tol=10e-7, n_iter_max=100, reg_W=1, verbose=0)
estimator1.fit(X, y1)
estimator2 = KruskalRegressor(weight_rank=rank, tol=10e-7, n_iter_max=100, reg_W=1, verbose=0)
estimator2.fit(X, y2)

(3000, 96, 55)
(3000,)


### latent factors

In [15]:
res1 =estimator1.kruskal_weight_
electro_latent1 = T.to_numpy(res1[1])

res2 =estimator2.kruskal_weight_
electro_latent2 = T.to_numpy(res2[1])


In [16]:
v = estimator1.weight_tensor_
v.shape

(96, 55)

In [ ]:
print(electro_latent1.shape)
print(electro_latent2.shape)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
fig = plt.figure(1)
ax = fig.add_subplot(1, 1,1)
ax.scatter(electro_latent[:,0],electro_latent[:,1], np.linspace(0,96),cmap=plt.cm.OrRd)
ax.set_axis_off()

fig2= plt.figure(2)
plt.plot(electro_latent[:,0])
plt.plot(electro_latent[:,1])

### Tensor prediction

In [ ]:
labels_test = estimator.predict(test_tensor)

In [ ]:
print(labels_test)
print(test_targets)

### Visualize Learned weights

In [ ]:
v= estimator.weight_tensor_
print(v.shape)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.imshow(T.to_numpy(estimator.weight_tensor_), cmap=plt.cm.OrRd, interpolation='nearest')
ax.set_axis_off()
ax.set_title('Learned regression weights')

In [ ]:
v = f[x_ref[6,0,0]][()]
d = v[0:3000,:]
d.shape

In [ ]:
neuron_tensor = np.zeros([96,3000,40]);
v = np.ones([96,3000]);
v.shape

neuron_tensor[:,:,0] = v;
neuron_tensor[:,:,0]


In [ ]:
a = np.random.rand(3,3,3)
x = T.tensor(a)
print(x)
print(a)

### Few Questions:

1. Non-negative tensor decomposition using tensorly?
2. Different kinds of decomposition - Tucker/CP/non-negative?
3. Tensor regression assumes each trial is independent (or each time-point is sampled iid). [NOT TRUE!]
4. Tensor based Gaussian processes (for smoothing the spike-bins)?!